In [24]:
from qiskit.transpiler import CouplingMap
from qiskit.transpiler.passes import SabreLayout, SabreSwap
from utils import file_to_coupling_map, directory_to_circuits, build_pass_manager
import random
import pandas as pd
import time


In [26]:
# Set random seed for reproducibility
seed = 42
random.seed(seed)
transpilation_trials = 10
seed_list = random.sample(range(0, 100000), transpilation_trials) 

# Load the coupling map and the quantum circuit
cm_name = 'heavy_hex_127'
qc_name = 'ghz'
cm = CouplingMap(file_to_coupling_map(f'coupling_maps/{cm_name}.txt'))
qc_list = directory_to_circuits(f'{qc_name}')

# Create the pass manager
layout_trials = 10
swap_trials   = layout_trials # same number of trials for layout and swap
heuristic     = "lookahead"
rp  = SabreSwap(coupling_map=cm, heuristic=heuristic, seed = seed_list[0], trials=swap_trials)
lp  = SabreLayout(coupling_map=cm, routing_pass=rp, seed=seed_list[1])
pm = build_pass_manager(cm, lp, rp)

# One pass trial
qc = qc_list[0]

time_start = time.time()
qc_tr = pm.run(qc)
time_end = time.time()

runtime = time_end - time_start
swap_count = qc_tr.count_ops()['swap']
depth = qc_tr.depth(lambda x: x.operation.num_qubits == 2) # 2 qubit gates
print(f"Runtime: {runtime:.10f} seconds")
print(f"Swap count: {swap_count}")
print(f"Depth: {depth}")



Runtime: 0.1054739952 seconds
Swap count: 16
Depth: 25
